# Imports

In [1]:
import os
import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
from pathlib import Path
import yaml
import argparse

from torch.utils.data import DataLoader, TensorDataset

# In script
# from mtl import data_setup, engine, models, utils

os.makedirs(os.getcwd() +'/mtl', exist_ok=True)

# Plotting

In [7]:
%%writefile mtl/plotting.py

from pathlib import Path
import matplotlib.pyplot as plt
from typing import Dict, List
import torch
from torch import nn
import os
import pandas as pd
import numpy as np
import yaml

from mtl import utils

# def plot_loss_curves_wrapper(directory: str, 
#                         mode: str, 
#                         y_lim=[-0.1, 10.0])-> None:
#     from mtl import utils
    
#     files = [f for f in os.listdir(directory) if f.endswith('.csv')]
#     betas = [files[i].replace('.csv', '') for i in range(len(files))]
    
#     # For plotting
#     for i in range(len(files)):
#         history = {}
#         df = pd.read_csv(directory + f'{files[i]}')
#         history = df.to_dict(orient='list')
#         history.pop('', None)
#         try:
#             history.pop('Unnamed: 0')
#         except: pass
#         beta = betas[i]
#         utils.plot_loss_curves(history, y_lim=[y_lim[0], y_lim[1]], mode=mode, beta=beta)
#     return None

def plot_loss_curves(results: Dict[str, List[float]],
                     y_lim = [-0.1, 10.],
                     beta=0., 
                     mode='test'):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
        beta : current beta value for plot title
        y_lim : Upper limit for plots
        mode : 'test_only' or 'all'
    
    
    Usage:     
        directory = os.getcwd() + os.path.join('/history/', timestamp, experiment_name, extra)
        
        files = [f for f in os.listdir(directory) if os.path.isfile(directory + f)]
        betas = [files[i].replace('.csv', '') for i in range(len(files))]
        
        # For plotting
        for i in range(len(files)):
            history = {}
            df = pd.read_csv(directory + f'{files[i]}')
            history = df.to_dict(orient='list')
            history.pop('', None)
            history.pop('Unnamed: 0')
            beta = betas[i]
            
            utils.plot_loss_curves(history, y_lim=0.4, mode='test', beta=beta)      

    """
    
    import matplotlib.pyplot as plt
    import numpy as np

    if mode == 'all':
        # Get the loss values of the results dictionary (training and test)
        train_loss_main = results['train_loss_main']
        train_loss_tot = results['train_loss_tot']
        train_loss_aux = results['train_loss_aux']
        test_loss_main = results['test_loss_main']

        # Figure out how many epochs there were
        epochs = range(len(results['train_loss_main']))

        # Setup a plot 
        plt.figure(figsize=(20, 8))

        # Plot loss
        plt.subplot(1, 2, 1)
        plt.plot(epochs, train_loss_main, label='train_loss_main')
        plt.plot(epochs, train_loss_aux, label='train_loss_aux')
        plt.plot(epochs, train_loss_tot, label='train_loss_tot')
        plt.ylim(y_lim[0], y_lim[1])
        plt.xticks(np.arange(0, len(epochs), 10))
        #plt.yticks(np.linspace(min(train_loss_tot), max(train_loss_tot)) )
        #plt.tick_params(left = False, labelleft = False) 
        
        #plt.title('Train Losses, Beta = ' + beta)
        plt.title(f'Train Losses {beta}')

        plt.xlabel('Epochs')
        plt.grid()
        plt.legend()
        
        # Plots main loss on train and test
        plt.subplot(1, 2, 2)
        plt.plot(epochs, train_loss_main, label='train_loss_main')
        plt.plot(epochs, test_loss_main, label='test_loss_main')
        plt.xticks(np.arange(0, len(epochs), 10))
        plt.ylim(y_lim[0], y_lim[1])
        #plt.tick_params(left = False, labelleft = False) 
        
        plt.title(f'Main Losses, {beta}')
        plt.xlabel('Epochs')
        plt.grid()
        plt.legend()
        
    elif mode=='test':
        # Get the loss values of the results dictionary (training and test)
        train_loss_main = results['train_loss_main']
        test_loss_main = results['test_loss_main']

        # Figure out how many epochs there were
        epochs = range(len(results['train_loss_main']))

        # Setup a plot 
        plt.figure(figsize=(8, 5))
        
        # Plots main loss on train and test
        plt.plot(epochs, train_loss_main, label='train_loss_main')
        plt.plot(epochs, test_loss_main, label='test_loss_main')
        plt.ylim(y_lim[0], y_lim[1])
        #plt.tick_params(left = False, labelleft = False) 
        plt.xticks(np.arange(0, len(epochs), 10))

        plt.title(f'Main Losses, {beta}')
        plt.xlabel('Epochs')
        plt.grid()
        plt.legend()
        
# def plot_adv_curves(adv_dict: Dict[str, List[float]], mode='adv', 
#                     x_lim = [0., 200.], y_lim = [-0.1, 10.], save=False, save_name=None):
#     ''' 
#     Args:
#         * Path to history folder
#         * mode: 'adv' for inverse advantage, 
#                 'std' for Test_loss(beta), 
#                 'vs' for L_alpha vs L_0
#             gets called in utils.adv_from_csv(history_path, mode=mode)
#         * y_lim: upper bound for plot
#     Returns: None
#     Plots: Advantage score curves vs Epochs for different betas
#     '''

#     import numpy as np
#     import os
#     from mtl_modular import utils
#     import matplotlib.pyplot as plt
#     import os
#     import pandas as pd
#     import warnings
#     warnings.simplefilter("ignore")
    
#     from datetime import datetime
#     import os
#     from pathlib import Path
#     from mtl_modular import utils

#     epochs = np.arange(0., 500, 1)

#     plt.figure(figsize=(10, 8))
#     plt.title(f'Learning Curves')
#     plt.xticks(np.arange(0, len(epochs), len(epochs)//10))
#     plt.xlim(x_lim[0], x_lim[1])
#     plt.ylim(y_lim[0], y_lim[1])
#     plt.grid()
#     plt.xlabel('Epochs')
#     plt.ylabel('Test Loss')
    
#     for key in adv_dict.keys():
#         ll = len(adv_dict[key])
#         plt.plot(epochs[:ll], adv_dict[key], label= f'{key}')
#         plt.legend()
        
#     if save == True:
#         if save_name is not None:
#             plt.savefig(f'{history_path}{mode}_{save_name}.png', bbox_inches='tight')
#         else:
#             plt.savefig(f'{history_path}{mode}.png', bbox_inches='tight')


def plot_adv_beta(adv_dicts, labels,
                  y_lim=None, plot=True, save=False,
                  directory=None, save_name=None, mode='adv', ylabel=None):

    ''' 
    Args: 
        - adv_dicts : List of Dictionaries out of adv_from_csv function
        - y_lim = None: upper y limit in plot
        - plot = True : bool, whether to display plot
        - save = False : bool, whether to save plot
        - directory = None : str, where plot will be saved
        - save_name = None : str, filename of png image
        - mode = 'adv': 'adv' or 'loss' : label of y axis
     
    Returns: 
        - Dictionary: key = beta : value = advantage score
        
    Plots: 
        - Final Advantage score/Loss score vs Beta
    '''
#    CONFIG_PATH = os.getcwd() + '/config'
#    config = load_config(path=CONFIG_PATH, config_name='1.yaml')
#    N = config['N']
#    DIM = config['DIM']
#    hiddens = config['HIDDENS']
#    title = f'Data Complexity: {len(hiddens)}, Data shape: [{N}, {DIM}], Random Seed: {seed} '

    #plt.figure(figsize=(16, 8))
    plt.grid()
    
    titles = ['Adv', 'Loss']
    #colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, 20) )
    
    for i, adv_dict in list(enumerate(adv_dicts)):
        best_list = [] ## Needs to be single value
        betas = []
        for beta in adv_dict.keys():
            best_list.append(adv_dict[beta])
            if beta not in betas:
                betas.append(float(beta)) 
        
        if plot:
            plt.ylim(-0.0005, y_lim)
            plt.plot(betas, best_list, label=labels[i])#, color=colors[i])
            plt.scatter(x=betas, y=best_list) #, label=labels[i])
            #print(f'betas = {betas}')
            plt.xticks(np.linspace(0, betas[-1], num=10))
            plt.title(f'{save_name}', fontsize=20)
            plt.xlabel('Beta')

            
                
            if mode=='adv':
                plt.ylabel(ylabel)
                plt.axhline(y=1., color='black', ls='--', lw=2.)
            elif mode=='loss':
                plt.ylabel('Loss')
            plt.legend()
            
            #Da capire : manca history_path per settare titolo e cartella di destinazione
            if save == True:
                if save_name is not None:
                    #plt.savefig(f'{history_path}{mode}_{save_name}.png', bbox_inches='tight')
                    plt.savefig(f'{directory}_{save_name}.png', bbox_inches='tight')
                else:
                    #plt.savefig(f'{history_path}{mode}.png', bbox_inches='tight')
                    print('Please, provide .png filename')
            else: pass
        
    out = dict((key, value) for key, value in zip(betas, best_list))
    return out



def plot_learning_curves(*curves, ylim1=-0.1, ylim2=3.):
    '''
    Use [] in place of missing lists to ignore
    '''
    plt.figure(figsize=(16, 8))

    for i, curve in list(enumerate(curves)):
        plt.plot(curve, label=str(i+1))
        
    plt.ylim(ylim1, ylim2)
    plt.legend()
    plt.grid()
    plt.show


def boxplot(subs_directory: str, metric: str, early_stopping=False) -> pd.DataFrame:

    '''
    Capisce file giusto in base a metric
    '''

    if metric not in ['MSE', '%Err', 'Pearson']:
        raise Exception("metric should be one of 'MSE', '%Err', 'Pearson'")

    if early_stopping:
        if metric=='MSE':
            file_name = 'Loss_scores_early_stopping.csv'
            y_label = r'$\mathscr{L}$'
        elif metric=='%Err':
            file_name = '%Error_on_variance_early_stopping.csv'
            y_label = '%Err on var(y)'
        elif metric=='Pearson':
            file_name = 'Pearson_early_stopping.csv'
            y_label = r'$\rho$'
            
    else:
        if metric=='MSE':
            file_name = 'Loss_scores_val_file.csv'
            y_label = r'$\mathscr{L}$'
        elif metric=='%Err':
            file_name = '%Error_on_variance_val_file.csv'
            y_label = '%Err on var(y)'
        elif metric=='Pearson':
            file_name = 'Pearson_val_file.csv'
            y_label = r'$\rho$'

    csv_file_path = subs_directory + file_name
    df = pd.read_csv(csv_file_path)
    keys = df.columns.tolist()

    boxplot_y = [df[key].tolist() for key in keys]

    means = [np.mean(boxplot_y[i]) for i in range(len(boxplot_y))]
    std_devs = [np.std(boxplot_y[i]) for i in range(len(boxplot_y))]
    
    # Box plot
    plt.figure(figsize=(10, 6))
    plt.boxplot(boxplot_y, labels=keys, patch_artist=True, boxprops=dict(facecolor='LightGreen'))
    plt.axhline(means[0], color='red', ls='--', lw=.8)

    # plt.yscale('symlog')

    plt.xlabel(r'$\beta$',  fontsize=16)
    plt.ylabel(y_label,  fontsize=16)

    # plt.axhline(y=0.953, color='red', ls='--', lw=1.7)
    # plt.annotate('std', xy=(0, 0.953) , xytext=(-0.17, 0.953), annotation_clip=False, color='red')
    # plt.xticks(np.linspace(0, keys[-1], num=10))

    plt.title(f'{y_label} vs '+ r'$\beta$')
    plt.grid()
    # plt.show()

    return df


def errorplot(subs_directory: str, 
                metric: str) -> pd.DataFrame:

    if metric not in ['MSE', '%Err', 'Pearson']:
        raise Exception("metric should be one of 'MSE', '%Err', 'Pearson'")

    if metric=='MSE':
        file_name = 'Loss_scores_val_file.csv'
        y_label = r'$\mathscr{L}$'
    elif metric=='%Err':
        file_name = '%Error_on_variance_val_file.csv'
        y_label = '%Err on var(y)'
    elif metric=='Pearson':
        file_name = 'Pearson_val_file.csv'
        y_label = r'$\rho$'

    csv_file_path = subs_directory + file_name
    df = pd.read_csv(csv_file_path)
    keys = df.columns.tolist()

    boxplot_y = [df[key].tolist() for key in keys]
    
    means = [np.mean(boxplot_y[i]) for i in range(len(boxplot_y))]
    std_devs = [np.std(boxplot_y[i]) for i in range(len(boxplot_y))]
    
    # advvv = [means[i]/means[0] for i in range(len(means))]
    # err_advvv = [(  np.sqrt( (np.square(means[i]*std_devs[0]) / (np.power(means[0], 4))) + np.square( std_devs[i] / means[0]  )  )  ) for i in range(len(means))]

    plt.figure(figsize=(10, 6))
    # Loss
    plt.errorbar(keys, means, yerr=std_devs, fmt='o', capsize=5, markersize=3)
    plt.axhline(y=means[0], color='red', ls='--', lw=1.)    

    plt.xlabel(r'$\beta$',  fontsize=18)
    plt.ylabel(y_label,  fontsize=18)
    plt.title(y_label + r'  vs  $\beta$')  

    # Adv
    # plt.errorbar(keys, advvv, yerr=err_advvv, fmt='o', capsize=5, markersize=3)
    # plt.axhline(y=1., color='red', ls='--', lw=1.)    
    
    # plt.xscale('symlog')
    # plt.yscale('symlog')  
    # plt.xticks([0, 0.2, 0.6, 1, 2, 3, 7, 10]) 
    # plt.yticks([0, 0.2, 0.5, 1, 2, 3, 4]) 
    # plt.ylim(ymin=0)
    # plt.ylabel('$Adv$',  fontsize=18)
    # plt.title('Adv vs Beta')

    plt.grid()
    plt.show()

    return df

Writing mtl/plotting.py


# Utils

In [8]:
%%writefile mtl/utils.py

from pathlib import Path
import matplotlib.pyplot as plt
from typing import Dict, List
import torch
from torch import nn
import os
import pandas as pd
import numpy as np
import yaml

history_path = os.getcwd() + '/history/'

def copy_file(yaml_file: str,
                target_dir_path: str,
              source=os.getcwd() + '/config/' ) -> None:
    '''
    Copies file from source to experiment folder specified by config file.
    
    Usage: copy_file(filename='my_config.yaml', 
              source=os.getcwd() + '/config/' )
    
    '''
    # import utils
    import shutil
    # import os
    # from datetime import datetime
    
    assert yaml_file.endswith(".yaml")
    # config = utils.load_config(path=source, config_name=yaml_file)
    
    # experiments_path = os.getcwd() + '/experiments'
    # timestamp = datetime.now().strftime("%m-%d-%Y") 
    # N = config['N']
    # DIM = config['DIM']
    # N_AUX = config['N_AUX']
    # EPOCHS = config['EPOCHS']
    # EXP = config['experiment_name']
    # SUBEXP = config['subexperiment_name']
    # directory = f'N={N}/DIM={DIM}/N_AUX={N_AUX}'

    # directory = os.path.join(experiments_path, timestamp, EXP, directory, SUBEXP)
    # target_dir_path = Path(directory)
    # target_dir_path.mkdir(parents=True,
    #                       exist_ok=True)
    
    shutil.copy(source + yaml_file, target_dir_path)
    return None

# Function to load yaml configuration file
def load_config(path, config_name):
    with open(os.path.join(path, config_name)) as file:
        config = yaml.safe_load(file)

    return config

def save_model(model: torch.nn.Module,
                target_dir_path: str,
                 model_name: str):

    """Saves a PyTorch model to a target directory.

    Args:
      model: A target PyTorch model to save.
      target_dir: A directory for saving the model to.
      model_name: A filename for the saved model. Should include
        either ".pth" or ".pt" as the file extension.
    
    Example usage:
      save_model(model=model_0,
                 CONFIG_PATH = os.getcwd() + '/config',
                 yaml_file='my_config.yaml'
                 model_name="05_going_modular_tingvgg_model.pth")
    """
    
    # from datetime import datetime
    # import utils

    # config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    # experiments_path = os.getcwd() + '/experiments'
    # timestamp = datetime.now().strftime("%m-%d-%Y") 
    # N = config['N']
    # DIM = config['DIM']
    # N_AUX = config['N_AUX']
    # EPOCHS = config['EPOCHS']
    # EXP = config['experiment_name']
    # SUBEXP = config['subexperiment_name']
    # directory = f'N={N}/DIM={DIM}/N_AUX={N_AUX}'

    # directory = os.path.join(experiments_path, timestamp, EXP, directory, SUBEXP,)
    # target_dir_path = Path(directory)
    # target_dir_path.mkdir(parents=True,
    #                       exist_ok=True)
    
    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path} \n")
    torch.save(obj=model.state_dict(),
               f=model_save_path)


def history_to_csv(history: dict, 
                    target_dir_path: str,
                    model_name: str) -> None:
    
    """Saves training history to a specific directory.
    Input: 
            * experiment_name (same as in 'runs' directory)
            * model_name may be some architecture characteristic (es. # auxiliary tasks or beta value) --> so far it's BETA
            * extra may be additional info
            
    Usage:  utils.history_to_csv(history=history, 
                    target_dir_path = utils.path_handler(CONFIG_PATH = os.getcwd() + '/config', yaml_file='my_config',)
                    model_name=f'Beta={BETAS[1]:.2f}')


    """
    # from datetime import datetime
    # import os
    # from pathlib import Path
    # import utils
    
    # # Config
    # config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)

    # # Create target directory
    # experiments_path = os.getcwd() + '/experiments'
    
    # # Get timestamp of current date in reverse order
    # timestamp = datetime.now().strftime("%m-%d-%Y") 
    # N = config['N']
    # DIM = config['DIM']
    # N_AUX = config['N_AUX']
    # EPOCHS = config['EPOCHS']
    # EXP = config['experiment_name']
    # SUBEXP = config['subexperiment_name']
    # directory = f'N={N}/DIM={DIM}/N_AUX={N_AUX}'

    # directory = os.path.join(experiments_path, timestamp, EXP, directory, SUBEXP)

    hist_df = pd.DataFrame(history)
    hist_csv_file = target_dir_path / f'{model_name}.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f, index=False)
            
    print(f"[INFO] Created .csv file to {target_dir_path}/{model_name}.csv")
    return None


def path_handler(CONFIG_PATH: str, yaml_file: str ) -> str:
    from datetime import datetime
    import os
    from pathlib import Path
    # from mtl import utils
    
    def load_config(path, config_name):
        with open(os.path.join(path, config_name)) as file:
            config = yaml.safe_load(file)
        return config

    # config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    config = load_config(path=CONFIG_PATH, config_name=yaml_file)

    experiments_path = os.getcwd() + '/experiments'
    timestamp = datetime.now().strftime("%m-%d-%Y")

    N = config['N']
    DIM = config['DIM']
    N_AUX = config['N_AUX']
    EPOCHS = config['EPOCHS']
    EXP = config['experiment_name']
    SUBEXP = config['subexperiment_name']
    ANTAGONIST = config['ANTAGONIST']
    directory = f'N={N}/DIM={DIM}/N_AUX={N_AUX}/ANTAGONIST={ANTAGONIST}'

    directory = os.path.join(experiments_path, timestamp, EXP, directory, SUBEXP)
    target_dir_path = Path(directory)
    target_dir_path.mkdir(parents=True,
                          exist_ok=True)
    return target_dir_path


##################################################################################################################
class Advantage:
    '''
    Usage: 
    import os 
    from mtl_modular import utils
    directory = os.getcwd() + f'/experiments/07-02-2024/exp_113/sub_1/N=1000/DIM=20/N_AUX=1/EPOCHS=500/'
    adv = Advantage() # Instantiate class
    bb = adv.from_csv(path=directory) # Returns loss curves
    best = adv.take_best(adv_dict=bb)
    adv.compute_adv(best)
    
        OR
        
    import os 
    from mtl_modular import utils
    directory = os.getcwd() + f'/experiments/07-02-2024/exp_113/sub_1/N=1000/DIM=20/N_AUX=1/EPOCHS=500/'
    adv = utils.Advantage()
    scores = adv.compute_adv(adv.take_best(adv_dict=adv.from_csv(path=directory)))
    utils.plot_adv_beta(scores)
    '''
    
    def __init__(self):
        return None
    
    
    def from_csv(self, path=None, 
                mode='test'):
        ''' 
        Arguments:
        * PATH storing csv files. 
            directory = os.getcwd() + f'/experiments/05-02-2024/exp_5/sub_1/N=1000/DIM=20/N_AUX=1/EPOCHS=300/'
            - mode: 'test': only keeps test loss
                    
                    'all': keeps all training history
                
        Returns: dictionary -> Beta : testing loss curve of training run
                                lenght is equal to training EPOCHS
    
        May be used with utils.plot_adv_beta as:
                        utils.plot_adv_beta(utils.from_csv(directory))
        '''
    

        files = [f for f in os.listdir(path) if f.endswith('.csv')]
        keys = [files[i].replace('.csv', '') for i in range(len(files))]
        keys = [keys[i].replace('Beta=', '') for i in range(len(keys))]
        values = []
        columns = ['test0']
        # if mode=='test':
        #     columns = ['test_loss_main']
        # elif mode=='all':
        #     columns = ['train_loss_main','train_loss_tot','train_loss_aux','test_loss_main','test_loss_aux']
        for i in range(len(files)):
            df = pd.read_csv(path + f'{files[i]}', usecols=columns)
            hhh = df.to_dict(orient='list')
            # print(hhh)
            # Only need test_loss
            hhh_list = hhh['test0']
            # print(hhh_list)
            # Returns loss
            values.append(hhh_list)
        out_dict = dict((key, value) for key, value in zip(keys,values))
        return out_dict   
    

    def take_best(self, adv_dict: Dict[str, List[float]]):
        keys = list(adv_dict.keys())
        values = [min(adv_dict[key]) for key in adv_dict.keys()]
        return dict((key, value) for key, value in zip(keys, values))
    
    def take_last(self, adv_dict: Dict[str, List[float]]):
        keys = list(adv_dict.keys())
        values = [adv_dict[key][-1] for key in adv_dict.keys()]
        return dict((key, value) for key, value in zip(keys, values))

    def compute_adv(self, adv_dict: Dict[str, List[float]]):
        keys = list(adv_dict.keys())
        #print(adv_dict[keys[0]])
        try:
            L0 = adv_dict['0.00']
        except: 
            print('L0 cannot be retrieved from file')
        values = [adv_dict[key]/L0 for key in adv_dict.keys()]
        return dict((key, value) for key, value in zip(keys, values))
    
    
    def runs_to_dict(self, subs_directory):
        '''
        Retrieves best score for loss and computes advantage score for each run and returns full dictionary
        Args:
         - subs_directory : experiment folder path -> subs should be equivalent runs when using this function.
        Returns:
         - Dictionary -> result_dict, loss_dict
                     { BETA: [adv_run1, adv_run2, ..] },  { BETA: [loss_run1, loss_run2, ..] }
        Usage: 
        
                import os
                from mtl_modular import utils
                
                path = os.getcwd() + f'/experiments/08-02-2024/exp_115/'
                adv = utils.Advantage()
                rr = adv.runs_to_dict(path)
            
                #To get mean and std, use:
                compute_stats(rr)
        '''
        adv_list = []
        loss_list = []
    
        for dirpath, dirnames, filenames in os.walk(subs_directory):
            if not dirnames:
                losses = self.take_last(adv_dict=self.from_csv(path=dirpath + '/'))
                loss_list.append(losses)
                scores = self.compute_adv(losses)
                adv_list.append(scores)
        
        adv_dict = {}
        loss_dict = {}
        
        for my_dict in adv_list:
            for key, value in my_dict.items():
                if key in adv_dict:
                    adv_dict[key].append(value)
                else:
                    adv_dict[key] = [value]
                    
        for my_dict in loss_list:
            for key, value in my_dict.items():
                if key in loss_dict:
                    loss_dict[key].append(value)
                else:
                    loss_dict[key] = [value]

        return adv_dict, loss_dict

    def compute_stats(self, result_dict):    
        '''
        Compute mean and standard deviation for each key, where values are lists.
        Args: 
            - Dictionary output of runs_to_dict function.
        Returns:
            - Dictionary -> { BETA: [mean, std_dev] }
        Usage: 
                import os
                from mtl_modular import utils
                path = os.getcwd() + f'/experiments/08-02-2024/exp_115/'
                adv = utils.Advantage()
                rr = adv.runs_to_dict(path)
            #To get mean and std, use:
                compute_stats(rr)
            
        '''
        # Compute mean and standard deviation for each key
        result_stats = {}
        for key, values in result_dict.items():
            mean_value = np.mean(values)
            std_deviation = np.std(values)
            # std_deviation = ( np.min(values) + np.max(values) )/2
            result_stats[key] = {'mean': mean_value, 'std': std_deviation}
        
        #for key, values in result_stats.items():
        #    print(key, values)
        return result_stats

################################################################################################################################

def get_data_distrib(subs_directory, plot=False):
    import os
    from mtl import utils
    import numpy as np
    import matplotlib.pyplot as plt
    from pathlib import Path
    import torch
    
    #directory = os.getcwd() + f'/experiments/29-02-2024/exp_prova/sub_9/N=45/DIM=20/N_AUX=1/EPOCHS=500/'
    #directory = os.getcwd() + f'/experiments/03-03-2024/exp_big_student/sub_1/N=10/DIM=20/N_AUX=1/EPOCHS=500/'

    for dirpath, dirnames, filenames in os.walk(subs_directory):
        # Check if the current directory has subdirectories
        if not dirnames:
            directory = dirpath
    
    yaml_file =  [f for f in os.listdir(directory) if f.endswith('.yaml')][0]
    config = utils.load_config(path=directory, config_name=yaml_file)
    
    C = len(config['HIDDENS'])
    S = len(config['S_HIDDENS'])
    C_list = list(config['HIDDENS'])
    S_list = list(config['S_HIDDENS'])

    N = config['N']
    DIM = config['DIM']
    N_AUX = config['N_AUX']
    
    print(f'Complexity: {C} {C_list} | Student: {S} {S_list}')

    all_data_path = os.getcwd() + '/data/'
    sub_path = f'DIM={DIM}/N_AUX={N_AUX}/C={C}/'
    data_dir = os.path.join(all_data_path, sub_path)
    #data_dir = os.getcwd() + f'/data/N={N}/DIM=20/N_AUX=1/C={C}/'
    
    print(f'Loading test data from :\n {data_dir}')
    train_data = torch.load(data_dir + 'test_data.pt')
    
    labels = []
    for X, y in train_data:
        labels.append(float(y[0].detach()))
    
    mean = np.mean(labels)
    std = np.std(labels)
    var = np.var(labels)
    if plot:
        plt.hist(labels, bins=100)
        plt.title(f'Mean = {mean:.3f} | Var = {var:.3f}')
        plt.grid()
        plt.show()
    else: pass
    print(f'Mean = {mean:.3f} | Var = {var:.3f}')
    return mean, var
    
### Usage:
# import os
# subs_directory = os.getcwd() + f'/data/DIM=20/N_AUX=1/C=1/'
# mean, var = get_data_distrib(subs_directory=subs_directory, plot=True)[1]




def exp_to_csv(subs_directory: str) -> None:
    '''
    Takes experiment path (with sub1, sub2, ..), creates and saves a csv file
    with rows = runs, columns = betas
    each entry is the last LOSS SCORE
    '''
    list_of_lists = []

    # Per ogni run
    for dirpath, dirnames, filenames in os.walk(subs_directory):
        if not dirnames:
            files = [f for f in os.listdir(dirpath) if f.endswith('.csv')]
            keys = [files[i].replace('.csv', '') for i in range(len(files))]
            keys = [keys[i].replace('Beta=', '') for i in range(len(keys))]
            keys = [float(key) for key in keys]

            loss_list = []

            # Per ogni beta
            columns = ['test0']
            for file in files:
                df = pd.read_csv(dirpath + f'/{file}', usecols=columns)
                l = df['test0'].iloc[-1]
                loss_list.append(l)

            # Ordering
            keys, losses = (list(t) for t in zip(*sorted(zip(keys, loss_list))))

            # list_of_lists[0] is the first run 
            list_of_lists.append(losses)

    df = pd.DataFrame(list_of_lists)
    df.columns = [str(key) for key in keys]
    df.head()

    # Loss_scores file
    hist_csv_file = subs_directory + 'Loss_scores_early_stopping.csv'
    with open(hist_csv_file, mode='w') as f:
        df.to_csv(f, index=False)

    print(f"[INFO] Created .csv file to {hist_csv_file}")
    return None


# def PearsonCoeff(true_labels, predictions):
#     import torch
#     pearson = torch.corrcoef(torch.stack([true_labels, predictions], dim=0))[0, 1].item()
#     return pearson

# def Error_on_variance(true_labels, predictions):
#     var = true_labels[:, 0].var().item()
#     error = torch.nn.MSELoss(true_labels, predictions) / var
#     return error

class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

class PearsonCoeff(torch.nn.Module):
    def __init__(self):
        super(PearsonCoeff, self).__init__()

    def forward(self, true_labels, predictions):
        pearson = torch.corrcoef(torch.stack([true_labels, predictions], dim=0))[0, 1]
        return pearson

class ErrorOnVariance(torch.nn.Module):
    def __init__(self):
        super(ErrorOnVariance, self).__init__()

    def forward(self, true_labels, predictions):
        var = true_labels.var().item()
        mse_loss = torch.nn.MSELoss()
        error = mse_loss(predictions, true_labels) / var
        return error

def advantage_error(means: List,
                    std_devs: List) -> List:
    '''
    Input : means = list of mean losses score
            std_devs = list of standard deviations for losses score
    Returns: err = list of error on advantage score
    Usage: adv_err = utils.advantage_error(means, std_devs)
    '''
    # err = [(  np.sqrt( (np.square(means[i]*std_devs[0]) / (np.power(means[0], 4))) + np.square( std_devs[i] / means[0]  )  )  ) for i in range(len(means))]
    # err = [np.sqrt( np.square( std_devs[i] / means[0]  ) + np.square(	(means[i]*std_devs[0]	/ np.power(means[0], 2)	)) for i in range(len(means))]
    err = [(means[i]/means[0]) * np.sqrt(  np.square(std_devs[i] / means[i]) + np.square(std_devs[0] / means[0])    ) for i in range(len(means))]
    return err


def evaluate_model(data_path: str, 
                    subs_directory: str,
                    yaml_file: str,
                    metric: str
                    ) -> List:

    '''
    Usage: 
        for N_AUX in range(1, 10):
            data_path = os.getcwd() + f'/data/DIM=20/N_AUX={N_AUX}/C=1'
            subs_directory = os.getcwd() + f'/experiments/04-20-2024/exp_aux{N_AUX}/'
            yaml_file = f'exp_aux{N_AUX}/1.yaml'
            utils.evaluate_model(data_path=data_path, subs_directory=subs_directory, yaml_file=yaml_file, boxplot=False)
    Returns: 
        Pandas DataFrame, columns: beta | rows: single runs | entries: Loss score
    '''
    if metric not in ['MSE', '%Err', 'Pearson', 'RMSE']:
        raise Exception("metric should be one of 'MSE', '%Err', 'Pearson', 'RMSE'")

    from mtl import models, plotting, utils
    import os 
    device = 'cpu'
    
    torch.manual_seed(3)

    # Data from file ----------------------------------
    test_data = torch.load(data_path + '/val_data.pt')
    X_test = test_data.tensors[0]
    y_test = test_data.tensors[1]
    # -------------------------------------------------

    student = models.Student(yaml_file=yaml_file)
    if metric=='MSE':
        loss_fn = torch.nn.MSELoss()
    elif metric=='%Err':
        loss_fn = utils.ErrorOnVariance()
    elif metric=='Pearson':
        loss_fn = utils.PearsonCoeff()
    elif metric=='RMSE':
        loss_fn = utils.RMSELoss()

    list_of_lists = []
    # Per ogni run
    for dirpath, dirnames, filenames in os.walk(subs_directory):
        if not dirnames:
            files = [f for f in os.listdir(dirpath) if f.endswith('.pth')]
            keys = [files[i].replace('.pth', '') for i in range(len(files))]
            keys = [keys[i].replace('Beta=', '') for i in range(len(keys))]
            keys = [float(key) for key in keys]
            keys, files = (list(t) for t in zip(*sorted(zip(keys, files))))

            # Per ogni beta
            losses = []
            for file in files:
                run = dirpath + '/' + file
                state_dict = torch.load(run,  map_location=torch.device('cpu'))
                student.load_state_dict(state_dict)

                with torch.inference_mode():
                    y_pred = student(X_test)
                    l = loss_fn(y_pred[:, 0], y_test[:, 0])
                    losses.append(l.item())
            list_of_lists.append(losses)

    df = pd.DataFrame(list_of_lists)
    df.columns = [str(key) for key in keys]

# csv file name
    if metric=='MSE':
        file_name = 'Loss_scores_val_file.csv'
    elif metric=='%Err':
        file_name = '%Error_on_variance_val_file.csv'
    elif metric=='Pearson':
        file_name = 'Pearson_val_file.csv'
    elif metric=='RMSE':
        file_name = 'RMSE_val_file.csv'


    hist_csv_file = subs_directory + file_name
    with open(hist_csv_file, mode='w') as f:
        df.to_csv(f, index=False)
    print(f"[INFO] Created .csv file to {hist_csv_file}")

    # out = [np.std(df[f'{key}']) for key in keys]
    # return out # standard dev across runs for fixed beta
    return df


def adv_and_err_from_df(df, beta, metric):
    '''
    Computes advantage and its error bar from full result dataframe
    Usage: df_adv, df_adv_err = utils.adv_and_err_from_df(df=df, beta=0.2)
    '''
    import pandas as pd

    df_beta = df.loc[df['BETA']==beta]
    df_0 = df.loc[df['BETA']==0.0]

    df_nn = df_beta.pivot('N', 'N_AUX', metric)
    df_nn0 = df_0.pivot("N", "N_AUX" , metric)

    df_nn_err = df_beta.pivot('N', 'N_AUX', 'STD_'+ metric)
    df_nn0_err = df_0.pivot('N', 'N_AUX', 'STD_'+ metric)

    df_adv = 1 - df_nn.div(df_nn0, axis=0)
    df_adv_err = df_nn.div(df_nn0) * np.sqrt(  ( df_nn_err.div(df_nn) )**2 + ( df_nn0_err.div(df_nn0) )  **2 )

    return df_adv, df_adv_err

def score_and_err_from_df(df, beta, metric):
    df_beta = df.loc[df['BETA']==beta]
    df_nn = df_beta.pivot('N', 'N_AUX', metric)
    df_nn_err = df_beta.pivot('N', 'N_AUX', 'STD_'+ metric)
    return df_nn, df_nn_err

Writing mtl/utils.py


# Data

In [3]:
# N_AUX = 3
# yaml_file = 'exp_aux3/1.yaml'
# N = 100
# DIM = 20
# teacher = models.Teacher(n_aux=N_AUX, yaml_file=yaml_file)#.to(device)
# # torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

# # train_data, test_data = data_setup.make_data(Teacher=teacher, N=N, dim=DIM)
# train, test = make_data(yaml_file=yaml_file)

# # train.tensors[0].shape, train.tensors[1].shape, test.tensors[0].shape, test.tensors[1].shape
# tt = test.tensors[1]
# print(tt)
# y_labels = tt[:, 0]
# print(y_labels)

In [ ]:
# import matplotlib.pyplot as plt
# tt = test.tensors[1]
# y_labels = tt[:, 0].numpy()
# aux_labels = tt[:, 1].numpy()

# # ------- Main ------------------------------
# y_mean = np.mean(y_labels)
# y_std = np.std(y_labels)
# y_var = np.var(y_labels)

# plt.hist(y_labels, bins=100)
# plt.title(f'Main task label distribution y\n\nMean = {y_mean:.3f} | Var = {y_var:.3f}')
# plt.grid()
# plt.show()
# print(f'Mean = {y_mean:.3f} | Var = {y_var:.3f}')

# # ------- Aux ------------------------------
# aux_mean = np.mean(aux_labels)
# aux_std = np.std(aux_labels)
# aux_var = np.var(aux_labels)

# plt.hist(aux_labels, bins=100)
# alpha = "\u03B1"
# plt.title(f'Aux task label distribution {alpha}\n\nMean = {aux_mean:.3f} | Var = {aux_var:.3f}')
# plt.grid()
# plt.show()
# print(f'Mean = {aux_mean:.3f} | Var = {aux_var:.3f}')

In [9]:
%%writefile mtl/data_setup.py
import os
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from pathlib import Path

try :
    import models, utils
except:
    from mtl import models, utils
    
def make_data(yaml_file: str):
    
    '''
    Generates 10.000 data points with labels given by teacher, normalizes 
    '''
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Config
    CONFIG_PATH = os.getcwd() + '/config'
    config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    DIM=config['DIM']
    N_AUX = config['N_AUX']
    experiment_name = config['experiment_name']

    torch.manual_seed(3)

    data_path = os.getcwd() + '/data/'
    sub_path = f'DIM={DIM}/N_AUX={N_AUX}'
    save_path = Path(os.path.join(data_path, experiment_name, sub_path))
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = str(save_path)

    # Actions
    X = torch.randn(104000, DIM, requires_grad=True).detach().to(device)
    teacher = models.Teacher(yaml_file=yaml_file).to(device)
    torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

    with torch.inference_mode():
        y = teacher(X).to(device)

    y_new = y.clone()
    for task in range(0, N_AUX+1):
      mean = torch.mean(y[:, task])
      std = torch.std(y[:, task])
      vals = (y[:, task] - mean) / std
      y_new[:, task] = vals

    X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=2000, random_state=42, shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2000, random_state=42, shuffle=True)

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    test_data = TensorDataset(X_test, y_test)
    
    return train_data, test_data, val_data, save_path


def make_data_noise(yaml_file: str):
    
    '''
    Generates 10.000 data points with labels given by teacher, normalizes 
    '''
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Config
    CONFIG_PATH = os.getcwd() + '/config'
    config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    DIM=config['DIM']
    N_AUX = config['N_AUX']
    NOISE = config['NOISE']
    experiment_name = config['experiment_name']

    torch.manual_seed(3)

    data_path = os.getcwd() + '/data/'
    sub_path = f'DIM={DIM}/N_AUX={N_AUX}/NOISE={NOISE}'
    save_path = Path(os.path.join(data_path, experiment_name, sub_path))
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = str(save_path)

    # Actions
    X = torch.randn(10000, DIM, requires_grad=True).detach().to(device)
    X_noise = X + torch.empty(10000, DIM).normal_(mean=0, std=NOISE) # vedi documentazione
    teacher = models.Teacher(yaml_file=yaml_file).to(device)
    torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')


    with torch.inference_mode():
      y = teacher(X).to(device)
      y_noisy = teacher(X_noise).to(device)

    yy = y[:, 0]
    mean = torch.mean(yy)
    std = torch.std(yy)
    vals = (yy - mean) / std
    yy = vals.unsqueeze(dim=1)

    aa = y_noisy[:, 1]
    mean = torch.mean(aa)
    std = torch.std(aa)
    vals = (aa - mean) / std
    aa = vals.unsqueeze(dim=1)

    y_new = torch.cat([yy, aa], dim=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=2000, random_state=42, shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2000, random_state=42, shuffle=True)

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    test_data = TensorDataset(X_test, y_test)
    
    return train_data, test_data, val_data, save_path


def make_data_noisy_labels(yaml_file: str):
    
    '''
    Generates 10.000 data points with labels given by teacher, normalizes 
    '''
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Config
    CONFIG_PATH = os.getcwd() + '/config'
    config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    DIM=config['DIM']
    N_AUX = config['N_AUX']
    NOISE = config['NOISE']
    experiment_name = config['experiment_name']

    torch.manual_seed(3)

    data_path = os.getcwd() + '/data/'
    sub_path = f'DIM={DIM}/N_AUX={N_AUX}/NOISE={NOISE}'
    save_path = Path(os.path.join(data_path, experiment_name, sub_path))
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = str(save_path)

    # Actions
    X = torch.randn(10000, DIM, requires_grad=True).detach().to(device)
    teacher = models.Teacher(yaml_file=yaml_file).to(device)
    torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

    with torch.inference_mode():
        y = teacher(X).to(device)

    y_new = y.clone()
    for task in range(0, N_AUX+1):
      mean = torch.mean(y[:, task])
      std = torch.std(y[:, task])
      vals = (y[:, task] - mean) / std
      y_new[:, task] = vals
      y_new[:, 1:] = y_new[:, 1:] + torch.empty(10000, 1).normal_(mean=0, std=NOISE)

    X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=2000, random_state=42, shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2000, random_state=42, shuffle=True)

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    test_data = TensorDataset(X_test, y_test)
    
    return train_data, test_data, val_data, save_path



def make_data_noisy_antagonist(yaml_file: str):
    '''
    Generates 10.000 data points with labels given by teacher, normalizes 
    '''
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Config
    CONFIG_PATH = os.getcwd() + '/config'
    config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
    DIM=config['DIM']
    N_AUX = config['N_AUX']
    NOISE = config['NOISE']
    ANTAGONIST = config['ANTAGONIST']
    experiment_name = config['experiment_name']

    torch.manual_seed(3)

    data_path = os.getcwd() + '/data/'
    sub_path = f'DIM={DIM}/N_AUX={N_AUX}/ANTAGONIST={ANTAGONIST}'
    save_path = Path(os.path.join(data_path, experiment_name, sub_path))
    save_path.mkdir(parents=True, exist_ok=True)
    save_path = str(save_path)

    # Actions
    X = torch.randn(100000, DIM, requires_grad=True).detach().to(device)
    teacher = models.Teacher(yaml_file=yaml_file).to(device)
    torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

    with torch.inference_mode():
        y = teacher(X).to(device)

    y_new = y.clone()
    for task in range(0, N_AUX+1):
      mean = torch.mean(y[:, task])
      std = torch.std(y[:, task])
      vals = (y[:, task] - mean) / std
      y_new[:, task] = vals
      if ANTAGONIST == 0:
        pass
      else:
        y_new[:, -ANTAGONIST:] = y_new[:, -ANTAGONIST:] + torch.empty(100000, 1).normal_(mean=0, std=NOISE)

    X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=2000, random_state=42, shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2000, random_state=42, shuffle=True)

    train_data = TensorDataset(X_train, y_train)
    val_data = TensorDataset(X_val, y_val)
    test_data = TensorDataset(X_test, y_test)
    
    return train_data, test_data, val_data, save_path


def make_dataloaders(
    train_data: torch.tensor, 
    test_data: torch.tensor,
    batch_size: int, 
    num_workers: int=0):
    
    """Creates training and testing DataLoaders.
    Args:
      train_data, test_data: torch.tensors (output of make_data)
      batch_size: Number of samples per batch in each of the DataLoaders.
      num_workers: An integer for number of workers per DataLoader.

    Returns:
      A tuple of (train_dataloader, test_dataloader).
    """
    
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=int(0),
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False, # don't need to shuffle test data
        num_workers=int(0),
        pin_memory=True,
    )

    return train_dataloader, test_dataloader

Writing mtl/data_setup.py


# Datasets

## create_dataset.py

In [10]:
%%writefile mtl/create_dataset.py

'''
# Command line arguments --> only config file (https://www.youtube.com/watch?v=FbEJN8FsJ9U)

Usage !python mtl/create_dataset.py exp_1/1.yaml
'''
import os
import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
from pathlib import Path
import yaml
import argparse

from mtl import data_setup, models, engine, utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Config
parser = argparse.ArgumentParser(description='specifies config file')
parser.add_argument('yaml_file', metavar='yaml_file', type=str, help='specify config file name with .yaml extension')
args = parser.parse_args()
yaml_file = args.yaml_file
CONFIG_PATH = os.getcwd() + '/config'
config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
N=config['N']
DIM=config['DIM']
N_AUX = config['N_AUX']
COMPLEXITY = len(config['HIDDENS'])
experiment_name = config['experiment_name']

# Save path
# timestamp = datetime.now().strftime("%m-%d-%Y") 

# data_path = os.getcwd() + '/data/'
# sub_path = f'DIM={DIM}/N_AUX={N_AUX}'#/C={COMPLEXITY}'
# save_path = Path(os.path.join(data_path, experiment_name, sub_path))
# save_path.mkdir(parents=True,
#                 exist_ok=True)
# save_path = str(save_path)

# Actions
teacher = models.Teacher(yaml_file=yaml_file).to(device)
# torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

train_data, test_data, val_data, save_path = data_setup.make_data(yaml_file=yaml_file)

torch.save(train_data, save_path + '/train_data.pt')
torch.save(test_data, save_path + '/test_data.pt')
torch.save(val_data, save_path + '/val_data.pt')

print(f'Datafiles saved to {save_path}')

Writing mtl/create_dataset.py


## create_dataset_noise.py

In [11]:
%%writefile mtl/create_dataset_noise.py

'''
# Command line arguments --> only config file (https://www.youtube.com/watch?v=FbEJN8FsJ9U)

Usage:
    import numpy as np
    noise_list = ['0.10', '0.20', '0.30', '0.40', '0.50', '0.60', '0.70', '0.80', '0.90', '1.00', '1.10']
    for NOISE in noise_list:
        !python mtl/create_dataset_noise.py exp_NOISE{NOISE}/1.yaml
'''
import os
import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
from pathlib import Path
import yaml
import argparse

from mtl import data_setup, models, engine, utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Config
parser = argparse.ArgumentParser(description='specifies config file')
parser.add_argument('yaml_file', metavar='yaml_file', type=str, help='specify config file name with .yaml extension')
args = parser.parse_args()
yaml_file = args.yaml_file
CONFIG_PATH = os.getcwd() + '/config'
config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
N=config['N']
DIM=config['DIM']
N_AUX = config['N_AUX']
COMPLEXITY = len(config['HIDDENS'])
experiment_name = config['experiment_name']

if N_AUX != 1:
    raise Exception("N_AUX should be 1")

# Save path
# timestamp = datetime.now().strftime("%m-%d-%Y") 

# data_path = os.getcwd() + '/data/'
# sub_path = f'DIM={DIM}/N_AUX={N_AUX}'#/C={COMPLEXITY}'
# save_path = Path(os.path.join(data_path, experiment_name, sub_path))
# save_path.mkdir(parents=True,
#                 exist_ok=True)
# save_path = str(save_path)
# print(save_path)

# Actions
# teacher = models.Teacher(yaml_file=yaml_file).to(device)
# torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

train_data, test_data, val_data, save_path = data_setup.make_data_noise(yaml_file=yaml_file)

torch.save(train_data, save_path + '/train_data.pt')
torch.save(test_data, save_path + '/test_data.pt')
torch.save(val_data, save_path + '/val_data.pt')

print(f'Datafiles saved to {save_path}')

Writing mtl/create_dataset_noise.py


## create_dataset_noisy_labels.py

In [12]:
%%writefile mtl/create_dataset_noisy_labels.py

'''
# Command line arguments --> only config file (https://www.youtube.com/watch?v=FbEJN8FsJ9U)

Usage:
    import numpy as np
    noise_list = ['0.01', '0.05', '0.10', '0.15','0.20', '0.30', '0.40', '0.50', '0.60', '0.70', '0.80', '0.90', '1.00', '1.10']
    for NOISE in noise_list:
        !python mtl/create_dataset_noisy_labels.py exp_NOISY_LABEL{NOISE}/1.yaml
'''
import os
import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
from pathlib import Path
import yaml
import argparse

from mtl import data_setup, models, engine, utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Config
parser = argparse.ArgumentParser(description='specifies config file')
parser.add_argument('yaml_file', metavar='yaml_file', type=str, help='specify config file name with .yaml extension')
args = parser.parse_args()
yaml_file = args.yaml_file
CONFIG_PATH = os.getcwd() + '/config'
config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
N=config['N']
DIM=config['DIM']
N_AUX = config['N_AUX']
COMPLEXITY = len(config['HIDDENS'])
experiment_name = config['experiment_name']

if N_AUX != 1:
    raise Exception("N_AUX should be 1")

# Save path
# timestamp = datetime.now().strftime("%m-%d-%Y") 

# data_path = os.getcwd() + '/data/'
# sub_path = f'DIM={DIM}/N_AUX={N_AUX}'#/C={COMPLEXITY}'
# save_path = Path(os.path.join(data_path, experiment_name, sub_path))
# save_path.mkdir(parents=True,
#                 exist_ok=True)
# save_path = str(save_path)
# print(save_path)

# Actions
# teacher = models.Teacher(yaml_file=yaml_file).to(device)
# torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

train_data, test_data, val_data, save_path = data_setup.make_data_noisy_labels(yaml_file=yaml_file)

torch.save(train_data, save_path + '/train_data.pt')
torch.save(test_data, save_path + '/test_data.pt')
torch.save(val_data, save_path + '/val_data.pt')

print(f'Datafiles saved to {save_path}')

Writing mtl/create_dataset_noisy_labels.py


## create_dataset_noisy_antagonist.py

In [13]:
%%writefile mtl/create_dataset_noisy_antagonist.py

'''
# Command line arguments --> only config file (https://www.youtube.com/watch?v=FbEJN8FsJ9U)

Usage:
    for i in range(10):
        !python mtl/create_dataset_noisy_antagonist.py exp_antagonist{i}/1.yaml
'''
import os
import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
from pathlib import Path
import yaml
import argparse

from mtl import data_setup, models, engine, utils

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Config
parser = argparse.ArgumentParser(description='specifies config file')
parser.add_argument('yaml_file', metavar='yaml_file', type=str, help='specify config file name with .yaml extension')
args = parser.parse_args()
yaml_file = args.yaml_file
CONFIG_PATH = os.getcwd() + '/config'
config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
N=config['N']
DIM=config['DIM']
N_AUX = config['N_AUX']
COMPLEXITY = len(config['HIDDENS'])
experiment_name = config['experiment_name']
ANTAGONIST = config['ANTAGONIST']

if N_AUX < ANTAGONIST:
    raise Exception("N_AUX < ANTAGONIST")

# Save path
# timestamp = datetime.now().strftime("%m-%d-%Y") 

# data_path = os.getcwd() + '/data/'
# sub_path = f'DIM={DIM}/N_AUX={N_AUX}/ANTAGONIST={ANTAGONIST}'
# save_path = Path(os.path.join(data_path, experiment_name, sub_path))
# save_path.mkdir(parents=True,
#                 exist_ok=True)
# save_path = str(save_path)
# print(save_path)

# Actions
# teacher = models.Teacher(yaml_file=yaml_file).to(device)
# torch.save(obj=teacher.state_dict(), f=save_path+'/teacher.pth')

train_data, test_data, val_data, save_path = data_setup.make_data_noisy_antagonist(yaml_file=yaml_file)

torch.save(train_data, save_path + '/train_data.pt')
torch.save(test_data, save_path + '/test_data.pt')
torch.save(val_data, save_path + '/val_data.pt')

print(f'Datafiles saved to {save_path}')

Writing mtl/create_dataset_noisy_antagonist.py


# Models

In [14]:
%%writefile mtl/models.py

import torch
from torch import nn
import yaml
import argparse 
import os
try:
    import utils
except: from mtl_modular import utils

class Teacher(nn.Module):
    def __init__(self, yaml_file):
        super().__init__()
        
        CONFIG_PATH = os.getcwd() + '/config/'
        config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
        
        self.hidden_sizes = config['HIDDENS']
        self.n_aux = config['N_AUX']
        self.dim = config['DIM']
        self.init = config['INIT']
        self.sigma = config['INIT_VALUE']

        act = config['ACTIVATION']
        if act == 'SIGMOID':
            self.activation = nn.Sigmoid()
        elif act == 'TANH':
            self.activation = nn.Tanh()
        elif act == 'RELU':
            self.activation = nn.ReLU()
        else: 
            raise Exception('[ERROR] Activation must be one of SIGMOID, TANH, RELU')

        # Layers
        self.hidden_layers = nn.ModuleList()
        self.aux_layers = nn.ModuleList()

        # Input Layer
        self.input_layer = nn.Linear(in_features=self.dim, out_features=self.hidden_sizes[0])
        self.hidden_layers.append(nn.LayerNorm(self.hidden_sizes[0]))
        self.hidden_layers.append(self.activation)

        # Hidden Layers
        for i in range(len(self.hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(self.hidden_sizes[i], self.hidden_sizes[i+1]))
            self.hidden_layers.append(nn.LayerNorm(self.hidden_sizes[i+1]))
            self.hidden_layers.append(self.activation)

        # Main Output Layer
        self.main_output_layer = nn.Linear(in_features=self.hidden_sizes[-1], out_features=1)
        
        # Auxiliary Output Layers
        for n in range(self.n_aux):
            self.aux_layers.append(nn.Linear(in_features=self.hidden_sizes[-1], out_features=1))        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            if self.init == 'UNIFORM':
                module.weight.data.uniform_(-self.sigma, +self.sigma)
            elif self.init == 'GAUSSIAN':
                module.weight.data.normal_(mean=0., std=self.sigma)
            else:
                raise Exception('[ERROR] INIT must be one of UNIFORM, GAUSSIAN.')
            
            if module.bias is not None:
                if self.init == 'UNIFORM':
                    module.bias.data.uniform_(-self.sigma, +self.sigma)
                elif self.init == 'GAUSSIAN':
                    module.bias.data.normal_(mean=0., std=self.sigma)
                else:
                    raise Exception('[ERROR] INIT must be one of UNIFORM, GAUSSIAN.')

    def forward(self, x):
        z = self.input_layer(x)
        for layer in self.hidden_layers:
            z = layer(z)
        
        # Main task
        y_main = self.main_output_layer(z)
        
        #Auxiliary tasks
        lista = []
        for layer in self.aux_layers:
            lista.append(layer(z))
        y_alpha = torch.cat(lista, dim=1)
        
        out = torch.cat([y_main, y_alpha], dim=1)
        return out
    
    
class Student(nn.Module):
    def __init__(self, yaml_file):
        
        CONFIG_PATH = os.getcwd() + '/config/'
        config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)
        
        super().__init__()
        self.hidden_sizes = config['S_HIDDENS']
        self.n_aux = config['N_AUX']
        self.dim = config['DIM']
        act = config['ACTIVATION']

        self.init = config['INIT']
        self.sigma = config['INIT_VALUE']
        self.norm = config['LAYER_NORM']

    
        if act == 'SIGMOID':
            self.activation = nn.Sigmoid()
        elif act == 'TANH':
            self.activation = nn.Tanh()
        elif act == 'RELU':
            self.activation = nn.ReLU()
        else: 
            print('[WARNING] Activation must be one of SIGMOID, TANH, RELU')
            pass
        
        # Layers
        self.hidden_layers = nn.ModuleList()
        self.aux_layers = nn.ModuleList()

        # Input Layer
        self.input_layer = nn.Linear(in_features=self.dim, out_features=self.hidden_sizes[0])
        if self.norm:
            self.hidden_layers.append(nn.LayerNorm(self.hidden_sizes[0]))
        self.hidden_layers.append(self.activation)
        
        # Hidden Layers
        for i in range(len(self.hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(self.hidden_sizes[i], self.hidden_sizes[i+1]))
            if self.norm:
                self.hidden_layers.append(nn.LayerNorm(self.hidden_sizes[i+1]))
            self.hidden_layers.append(self.activation)

        # Main Output Layer
        self.main_output_layer = nn.Linear(in_features=self.hidden_sizes[-1], out_features=1)
        
        # Auxiliary Output Layers
        for n in range(self.n_aux):
            self.aux_layers.append(nn.Linear(in_features=self.hidden_sizes[-1], out_features=1))        
        
        self.apply(self._init_weights)    
    

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            if self.init == 'UNIFORM':
                module.weight.data.uniform_(-self.sigma, +self.sigma)
            elif self.init == 'GAUSSIAN':
                module.weight.data.normal_(mean=0., std=self.sigma)
            else:
                raise Exception('[ERROR] INIT must be one of UNIFORM, GAUSSIAN.')
            
            if module.bias is not None:
                if self.init == 'UNIFORM':
                    module.bias.data.uniform_(-self.sigma, +self.sigma)
                elif self.init == 'GAUSSIAN':
                    module.bias.data.normal_(mean=0., std=self.sigma)
                else:
                    raise Exception('[ERROR] INIT must be one of UNIFORM, GAUSSIAN.')

    def forward(self, x):
        z = self.input_layer(x)
        for layer in self.hidden_layers:
            z = layer(z)
        
        # Main task
        y_main = self.main_output_layer(z)
        
        #Auxiliary tasks
        lista = []
        for layer in self.aux_layers:
            lista.append(layer(z))    
        y_alpha = torch.cat(lista, dim=1)
        
        out = torch.cat([y_main, y_alpha], dim=1)
        return out

Writing mtl/models.py


# Engine

In [15]:
%%writefile mtl/engine.py

import torch
from torch import nn
from typing import Dict, List, Tuple
# from tqdm.auto import tqdm

def train_step(model: nn.Module, 
                dataloader: torch.utils.data.DataLoader,
                loss_fn: nn.Module,
                loss_aux: nn.Module,
                optimizer: torch.optim.Optimizer, 
                device: torch.device, 
                BETAS: torch.tensor)-> Tuple[float, float]:
    
    model.to(device)
    model.train()
    BETAS.to(device)
    # print(f'BETAS: {BETAS}')

    loss_tracker = torch.zeros(1, len(BETAS))

    for batch, (X_,y_) in enumerate(dataloader):
        X_, y_ = X_.to(device), y_.to(device)
        preds = model(X_).to(device)

        lista = []
        for i in range(y_.shape[1]):
            lista.append(loss_aux(preds[:, i], y_[:, i]))
        
        # Tensor with each task loss as entry, first entry is MAIN TASK
        losses = torch.hstack(lista).to(device) 
        BETAS[0] = 1. # Main loss has fixed BETA
        
        # Weighted sum
        loss = torch.dot(losses, BETAS)

        # Loss tracking
        for i in range(loss_tracker.shape[1]):
            loss_tracker[:, i] += losses[i].item()

        # Backprop
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

    loss_tracker = loss_tracker / len(dataloader)
    
    return loss_tracker

    
def test_step(model: nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: nn.Module,
              device=torch.device) -> float:
    '''
    Only evaluate on main task
    '''
    model.to(device)
    model.eval()

    n_tasks = next(iter(dataloader))[1][0].shape[0]
    test_loss_tracker = torch.zeros(1, n_tasks)

    
    with torch.inference_mode():
        # Loop through batches
        for batch, (X_,y_) in enumerate(dataloader):
            # Data to device
            X_, y_ = X_.to(device), y_.to(device)
            
            preds = model(X_).to(device)

            lista = []
            for i in range(y_.shape[1]):
                lista.append(loss_fn(preds[:, i], y_[:, i]))

            losses = torch.hstack(lista).to(device) 

            # Loss tracking
            for i in range(test_loss_tracker.shape[1]):
                test_loss_tracker[:, i] += losses[i].item()

    test_loss_tracker = test_loss_tracker / len(dataloader)

    return test_loss_tracker

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer, 
          loss_fn: torch.nn.Module,
          loss_aux: torch.nn.Module, 
          epochs: int,
          BETAS: torch.tensor,
          patience: int,
          device: torch.device)  -> Dict[str, List[float]]:
    
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    - model: A PyTorch model to be trained and tested.
    - train_dataloader: A DataLoader instance for the model to be trained on.
    - test_dataloader: A DataLoader instance for the model to be tested on.
    - optimizer: A PyTorch optimizer to help minimize the loss function.
    - loss_fn: A PyTorch loss function to calculate loss on both datasets.
    - loss_aux: A PyTorch loss function for auxiliary tasks --> da rendere modificabile??
    - epochs: An integer indicating how many epochs to train for.
    - BETAS: Tensor containing beta values for different tasks.
    - device: A target device to compute on (e.g. "cuda" or "cpu").
    - writer: SummaryWriter instance to track result in tensorboard.

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss_main: [...],
              train_loss_tot: [...],
              train_loss_aux: [...],
              test_loss_main: [...]} 
    """
    
    results = {f'{key}{i}': []  for i in range(len(BETAS)) for key in ['train', 'test']}

    # Early Stopping       
    best_score = None
    #best_score_aux = None
    counter = 0
    patience = patience
        
    for epoch in range(1, epochs+1):
        # Initialization check
        if epoch==1:
            test_losses = test_step(model=model, 
                                            dataloader=test_dataloader, 
                                            loss_fn=loss_fn,
                                            device=device)                               
            print(f'test_loss_main: {test_losses[0, 0].item()} | test_loss_aux: {test_losses[0, 1].item()}')

        # Steps
        train_losses = train_step(model=model, 
                                               dataloader=train_dataloader, 
                                               loss_fn=loss_fn,
                                               loss_aux=loss_aux,
                                               optimizer=optimizer,
                                               device=device, 
                                                BETAS=BETAS)
        test_losses = test_step(model=model, 
                                        dataloader=test_dataloader, 
                                        loss_fn=loss_fn,
                                        device=device)
        # Printing message 
        if (epoch+1)%20 == 0:
            print(
              f"Epoch: {epoch+1} | "
              f"Betas: {[round(item, 2) for item in BETAS.tolist()]} | "
              f"train_loss_main: {train_losses[0, 0]:.4f} | "
              f"test_loss_main: {test_losses[0, 0]:.4f} | " 
              )
                
        # Fill return object
        for i in range(len(BETAS)):
            results[f'train{i}'].append(train_losses[0, i].item())
            results[f'test{i}'].append(test_losses[0, i].item())

        # Early Stopping
        test_loss_main = test_losses[0, 0].item()
        if best_score is None:
            best_score = test_loss_main
            #best_score_aux = test_loss_aux
        else:
            # MAIN LOSS
            if test_loss_main < best_score:
                # val_loss improves, we update the latest best_score, 
                best_score = test_loss_main
            else:
                # val_loss does not improve, we increase the counter, 
                # stop training if it exceeds the amount of patience
                counter += 1
                if counter >= patience:
                    print(f'[EARLY STOPPING] Epoch: {epoch} | Test Loss Main: {best_score}\n'
                    '-----------------------------------------------------------------------------')
                    break    
            
    return results

Writing mtl/engine.py


## Debugging Engine

In [17]:
# from mtl import data_setup

# data_path = '/mnt/c/Users/Cesare/Desktop/Tesi/code/data/DIM=20/N_AUX=3/C=1'

# train_data = torch.load(data_path + '/train_data.pt')
# test_data = torch.load(data_path + '/test_data.pt')

# X_train = train_data.tensors[0]
# y_train = train_data.tensors[1]

# X_test = test_data.tensors[0]
# y_test = test_data.tensors[1]

# # Create a new TensorDataset with the first N elements
# train_data = TensorDataset(X_train[:100], y_train[:100])
# test_data  = TensorDataset(X_test[:2000], y_test[:2000])


# train_dataloader, test_dataloader = data_setup.make_dataloaders(
#                                                         train_data = train_data, 
#                                                         test_data = test_data,
#                                                         batch_size=32,
#                                                         num_workers=4)
# train_dataloader
# n_tasks = next(iter(train_dataloader))[1][0].shape[0]
# n_tasks
# losses = torch.zeros(1, n_tasks)
# losses

# aa = torch.ones(1, n_tasks)

# j = torch.cat([losses, aa], dim=0)
# j
# loss_tracker = torch.zeros(1, n_tasks)
# loss_tracker.shape

# for i in range(loss_tracker.shape[1]):
#     loss_tracker[:, i] = i

# loss_tracker[0, 3].item()
# betasss  = torch.zeros(1, 3)
# betasss
# betasss.tolist()
# Train

In [18]:
%%writefile mtl/train0.py

'''
Usage : !python mtl_modular/train.py exp_1/my_config.yaml
'''

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
from torch import nn
from timeit import default_timer as timer
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.simplefilter('ignore')
from datetime import datetime
import yaml
import argparse

# In script
try:
    import data_setup, engine, models, utils
# In jupyter
except:
    from mtl import data_setup, engine, models, utils


# Command line arguments --> only config file (https://www.youtube.com/watch?v=FbEJN8FsJ9U)
parser = argparse.ArgumentParser(description='specifies config file')
parser.add_argument('yaml_file', metavar='yaml_file', type=str, help='specify config file name with .yaml extension')
args = parser.parse_args()
yaml_file = args.yaml_file

# folder to load config file
CONFIG_PATH = os.getcwd() + '/config'
config = utils.load_config(path=CONFIG_PATH, config_name=yaml_file)

# DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# DIRECTORIES
# history_path = config['history_path'] + config['experiment_name']
# saved_models_path = config['saved_models_path'] + config['experiment_name']
# images_path = config['images_path'] + config['experiment_name']

timestamp = datetime.now().strftime("%m-%d-%Y") 

# EXPERIMENT
experiment_name = config['experiment_name']

# DATA
all_data_path = os.getcwd() + '/data/'

N=config['N']
DIM=config['DIM']
N_AUX = config['N_AUX']
NOISE = config['NOISE']
print(f'NOISE = {NOISE}')

sub_path = f'DIM={DIM}/N_AUX={N_AUX}'
data_path = os.path.join(all_data_path, experiment_name, sub_path)

if 'nois' in experiment_name:
    data_path = data_path + f'/NOISE={NOISE}'

if 'antagonist' in experiment_name:
    data_path = data_path + f'/ANTAGONIST={ANTAGONIST}'

# antagonist and noise
if 'NOISE' in experiment_name:
    data_path = data_path + f'/ANTAGONIST={ANTAGONIST}'

# New runs to show MTL does not work
if 'different_teachers' in experiment_name:
    data_path = os.getcwd() + '/data/unrelated/different_teachers'

if 'randomized' in experiment_name:
    data_path = os.getcwd() + '/data/unrelated/randomized'

print(f'Data file from : \n {data_path}')

try: 
    NOISE = float(config['NOISE'])
except: pass
try:
    ANTAGONIST = config['ANTAGONIST']
except: pass

# HYPERPARAMETERS
RANDOM_SEED = config['RANDOM_SEED']
SAVE_MODE = config['SAVE_MODE']
ATTEMPTS = config['ATTEMPTS']
EPOCHS = config['EPOCHS']
BATCH_SIZE = config['BATCH_SIZE']
PATIENCE = config['PATIENCE']
OPTIMIZER = config['OPTIMIZER']
LR = float(config['LR'])

# Betas
BETA_START = config['BETA_START']
# MAX_BETA = config['MAX_BETA']
# dBETA = (MAX_BETA - BETA_START) / ATTEMPTS

# Beta tutti uguali
#BETAS = torch.zeros(N_AUX+1)
BETAS = torch.full(size=[N_AUX+1], fill_value=BETA_START).to(device)

# Beta random
#BETAS = torch.rand(N_AUX+1)

# start_time = timer()

# Load tensors and Create DataLoaders

train_data = torch.load(data_path + '/train_data.pt')
test_data = torch.load(data_path + '/test_data.pt')

X_train = train_data.tensors[0]
y_train = train_data.tensors[1]

X_test = test_data.tensors[0]
y_test = test_data.tensors[1]

# Create a new TensorDataset with the first N elements
train_data = TensorDataset(X_train, y_train)
dataset_size = N
indices = torch.randperm(dataset_size)
train_data = TensorDataset(X_train[indices], y_train[indices])

test_data  = TensorDataset(X_test[:2000], y_test[:2000])

train_dataloader, test_dataloader = data_setup.make_dataloaders(
                                                        train_data = train_data, 
                                                        test_data = test_data,
                                                        batch_size=BATCH_SIZE,
                                                        num_workers=4)


## ---------------- BETAS -------------------------------
# BETA_VALUES = np.logspace(-1, 1, num=21, endpoint=True) # Log space for [0, 10] interval
# BETA_VALUES = np.concatenate((np.zeros(1), BETA_VALUES), axis=0)

# BETA_VALUES = np.linspace(0, 1., num=11, endpoint=True) # Lin space for [0, 1.5] internal

BETA_VALUES = np.linspace(0, 1., num=11, endpoint=True)
bb = np.linspace(2., 10., num=9, endpoint=True)
BETA_VALUES = np.hstack([BETA_VALUES, bb])

# bb = np.linspace(2., 10., num=9, endpoint=True)
# BETA_VALUES = bb

## ------------------------------------------------------

for beta in BETA_VALUES:

    BETAS[1:] = beta
    try: del student
    except: pass
    
    # torch.manual_seed(RANDOM_SEED)
    # torch.cuda.manual_seed(RANDOM_SEED)
    
    student = models.Student(yaml_file=yaml_file).to(device)
    
    loss_main = nn.MSELoss()
    loss_aux = nn.MSELoss()

    if OPTIMIZER=='SGD':
        optimizer = torch.optim.SGD(params=student.parameters(), lr=LR)    
    elif OPTIMIZER=='ADAM':
        optimizer = torch.optim.Adam(params=student.parameters(), lr=LR, weight_decay=1e-4) # weight decay default
    else:
        raise Exception('[ERROR] Optimizer must be one of SGD, ADAM')


    history = []
    history = engine.train(model=student, 
                    train_dataloader=train_dataloader, 
                    test_dataloader=test_dataloader,
                    optimizer=optimizer, 
                    loss_fn=loss_main,
                    loss_aux=loss_aux, 
                    epochs= EPOCHS, 
                    BETAS=BETAS,
                    patience=PATIENCE,
                    device=device)
        
    if SAVE_MODE:
        save_path = utils.path_handler(CONFIG_PATH = os.getcwd() + '/config',  yaml_file=yaml_file)
        
        utils.history_to_csv(history=history, 
                            target_dir_path=save_path,
                            model_name=f'Beta={BETAS[1]:.2f}')
        
    # Saving model
        utils.save_model(model=student,
                        target_dir_path=save_path,
                        model_name=f'Beta={BETAS[1]:.2f}.pth')
        print('-----------------------------------------------------------------------------')
    else: pass
    
    
#    if BETAS[1] < 0.4:
#        BETAS[1:] += 0.1
#    elif 0.4 <= BETAS[1] <= 2.5:
#        BETAS[1:] += dBETA
#    else:
#        BETAS[1:] += 0.4

    # if BETAS[1] < 1.0:
    #     BETAS[1:] += 0.05

    # elif BETAS[1] < 2. :
    #     BETAS[1:] += 0.2

    # else: 
    #     BETAS[1:] += 2.


#    print(f'type Betas : {type(BETAS)}')
#    print(f'shape Betas : {BETAS.shape}')
#    print(f'Betas[0] : {BETAS[0]}')
#    print(f'Betas[1] : {BETAS[1]}')

        
utils.copy_file(yaml_file=yaml_file,
                target_dir_path=save_path,
                source=os.getcwd() + '/config/' )

Overwriting mtl/train0.py


# Main 

## Config

In [2]:
%%writefile mtl_modular/config_writer.py
import yaml
from pathlib import Path

def write_yaml_to_file(data, filename, experiment):
    directory = f'config/{experiment}/'
    directory = Path(directory)
    directory.mkdir(parents=True, exist_ok=True)
    
    with open(f'{directory}/{filename}.yaml', 'w',) as f :
        yaml.dump(data, f, sort_keys=False) 
    #print('Written to file successfully')

data = {
'data_path': '../data',
'history_path': '../history',
'saved_models_path': '../saved_models',
'images_path': '../images',
'experiment_name': 'exp_aux1',
'subexperiment_name' : 'sub_1',   
'HIDDENS': list([10]),
'S_HIDDENS': list([10]), 

'N': 1000,
'DIM': 20,
'RANDOM_SEED': 31,
'SAVE_MODE': True,
'ATTEMPTS': 30,
'EPOCHS': 500, 
'BATCH_SIZE': 32,
'PATIENCE' : 10, 
'N_AUX': 1,
'LR': '1e-2',
'OPTIMIZER' : 'ADAM',
'ACTIVATION': 'TANH',
'BETA_START': 0., 
'MAX_BETA': 11.
}
counter = int(1)
experiment = data['experiment_name']
subexperiment = data['subexperiment_name']

# ---------- single for debug ------------------------------
# write_yaml_to_file(data, f'{counter}', experiment)

# ---------- real exp --------------------------------------
# ---------- Varia N--------------------------------------

# for j in range(0, 2):
# RANGE_N = [100, 200, 500, 800, 1000, 1400, 1700, 2000]
# for N in RANGE_N:
#     counter = int(1)
#     data['N'] = N
#     # data['N'] += 500
#     # N = data['N']
#     data['experiment_name'] = f'exp_{N}'

#     for i in range(1, 20): 
#         data['subexperiment_name'] = f'sub_{i}'
#         experiment = data['experiment_name']
#         write_yaml_to_file(data, f'{counter}', experiment)
#         data['RANDOM_SEED'] += 5
#         counter+=1
    
for n_aux in range(5, 10, 1):
    counter = int(1)
    data['N_AUX'] = n_aux
    data['experiment_name'] = f'exp_aux{n_aux}'
    
    for i in range(1, 20): 
        data['subexperiment_name'] = f'sub_{i}'
        experiment = data['experiment_name']
        write_yaml_to_file(data, f'{counter}', experiment)
        data['RANDOM_SEED'] += 5
        counter+=1
    
print(f'Successfully written to config/{experiment}, {counter} files')

Overwriting mtl_modular/config_writer.py


In [3]:
!python mtl_modular/config_writer.py

Successfully written to config/exp_aux9, 20 files


## Dataset creation

In [4]:
for i in range(1, 10):
    !python mtl/create_dataset.py exp_aux{i}/1.yaml

Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=1/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=2/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=3/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=4/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=5/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=6/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=7/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=8/C=1
Datafiles saved to /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=9/C=1


## Training

In [ ]:
for i in range(1, 10):
    for j in range(1, 20):
        !python mtl/train.py exp_aux{i}/{j}.yaml

In [9]:
!python mtl/train.py exp_aux3/1.yaml

Data file from : 
 /home/cesare/TESI/Tesi/code/data/DIM=20/N_AUX=3/C=1
Epoch: 20 | Betas: [1.0, 0.0, 0.0, 0.0] | train_loss_main: 0.0137 | test_loss_main: 0.0165 | 
Epoch: 40 | Betas: [1.0, 0.0, 0.0, 0.0] | train_loss_main: 0.0070 | test_loss_main: 0.0105 | 
[EARLY STOPPING] Epoch: 42 | Test Loss Main: 0.009619008749723434
-----------------------------------------------------------------------------
[INFO] Created .csv file to /home/cesare/TESI/Tesi/code/experiments/04-20-2024/exp_aux3/sub_1/N=1000/DIM=20/N_AUX=3/EPOCHS=500/Beta=0.00.csv
[INFO] Saving model to: /home/cesare/TESI/Tesi/code/experiments/04-20-2024/exp_aux3/sub_1/N=1000/DIM=20/N_AUX=3/EPOCHS=500/Beta=0.00.pth 

-----------------------------------------------------------------------------
Epoch: 20 | Betas: [1.0, 0.1, 0.1, 0.1] | train_loss_main: 0.0097 | test_loss_main: 0.0135 | 
[EARLY STOPPING] Epoch: 36 | Test Loss Main: 0.006920445244759321
-----------------------------------------------------------------------------
[